# 1. Cython

Cython umožňuje psát kód podobný Pythonu, doplnit ho o statické typy a přeložit ho do C/C++ extension modulu.
To často přinese výrazné zrychlení numerických smyček.

Pro překlad je potřeba C/C++ kompilátor.

- Na Linuxu bývá dostupný přes balík `build-essential`.
- Na Windows je potřeba nainstalovat MSVC nástroje: <https://wiki.python.org/moin/WindowsCompilers>.

In [ ]:
#!pip install cython

## 1.1 Základní workflow

Pro jednoduchý modul obvykle potřebujeme:
- `.pyx` soubor se zdrojovým kódem,
- `setup.py` se specifikací překladu.

### 1.1.1 Soubor `.pyx`

In [ ]:
%%writefile hello_cython.pyx
print("Ahoj, Cythone!")

### 1.1.2 Soubor `setup.py`

In [ ]:
%%writefile setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules=cythonize("hello_cython.pyx")
)


`setup.py` popisuje, jak modul přeložit.

V této ukázce:
- `from setuptools import setup` načte build nástroje,
- `from Cython.Build import cythonize` přeloží `.pyx` do C a připraví extension,
- `setup(ext_modules=cythonize("hello_cython.pyx"))` spustí build modulu.

In [ ]:
!python setup.py build_ext --inplace

Po spuštění příkazu máme modul sestavený.

U příkazu `python setup.py build_ext --inplace`:
- `build_ext` spustí build extension modulů,
- `--inplace` uloží výstup přímo do aktuální složky.

Vyzkoušíme nově zkompilovaný modul v Pythonu:

In [ ]:
import hello_cython

Tím máme základní cyklus překladu ověřený.

## 1.2 Typování a práce s NumPy

Cython přidává explicitní typy, které pomáhají odstranit režii Python objektů v kritických částech kódu.

- `cdef` deklaruje C-level proměnné a funkce.
- `cpdef` vytvoří funkci dostupnou i z Pythonu.
- `cimport numpy as np` zpřístupní Cython typy pro NumPy pole.

U výkonných smyček se často používá:
- `@cython.boundscheck(False)` pro vypnutí kontroly mezí,
- `@cython.wraparound(False)` pro vypnutí záporného indexování.

Tyto optimalizace zapínejte až ve chvíli, kdy máte ověřenou správnost kódu.

## 1.3 Praktický příklad

In [ ]:
%%writefile my_dot.pyx
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython # je zde kvůli dekorátorům

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_dot(np.ndarray[np.float64_t, ndim=1] a, np.ndarray[np.float64_t, ndim=1] b):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * b[i]
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm(np.ndarray[np.float64_t, ndim=1] a):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * a[i]
    return sqrt(result)

- `cimport` je Cython obdoba `import` pro Cython definice.
- `libc` obsahuje vazby na C standardní knihovnu, např. `libc.math.sqrt`.

In [ ]:
%%writefile setup.py
from setuptools import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext_modules = [
    Extension(
        name = "my_dot",
        sources=["my_dot.pyx"],
        include_dirs=[np.get_include()],  # include path for NumPy headers
        extra_compile_args=["-O3"],  # optional, to enable optimization flags
    )
]

setup(
    ext_modules=cythonize(ext_modules),
)



Pro větší moduly je praktičtější použít `Extension`, kde explicitně nastavíme:
- jméno modulu,
- zdrojové soubory,
- include cesty (např. pro hlavičky NumPy),
- parametry kompilátoru (např. `-O3`).

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import my_dot

In [ ]:
import numpy as np
a = np.random.rand(2_000_000)
b = np.random.rand(2_000_000)

In [ ]:
%timeit c = my_dot.my_dot(a, b)

In [ ]:
%timeit c = np.dot(a, b)

In [ ]:
print(my_dot.my_dot(a, b), np.dot(a, b))

In [ ]:
%timeit c = my_dot.my_norm(a)

In [ ]:
%timeit c = np.linalg.norm(a)

## 1.4 Cython v prostředí Jupyter notebooků

Pro rychlé experimenty lze použít magic `%%cython` a psát Cython kód přímo do buňky bez ruční práce se soubory.

In [ ]:
# je třeba nejdříve načíst IPython extension pro Cython
%load_ext Cython

In [ ]:
%%cython --compile-args=-O3

import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython # je zde kvůli dekorátorům

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_dot2(np.ndarray[np.float64_t, ndim=1] a, np.ndarray[np.float64_t, ndim=1] b):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * b[i]
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm2(np.ndarray[np.float64_t, ndim=1] a):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * a[i]
    return sqrt(result)


In [ ]:
%timeit c = my_dot2(a, b)

In [ ]:
%timeit c = my_norm2(a)

## 1.5 Co dál

Cython umí výrazně víc než jen jednoduché funkce, například C++ integraci, šablony, OpenMP nebo tvorbu C-API.